In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import make_scorer, mean_absolute_error, r2_score
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
import lightgbm as lgb
import shap
import lime
import dask.dataframe as dd


e:\User\miniconda3\envs\enefit\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# from dask.distributed import Client
# client = Client(memory_limit='10GB')  # Tăng giới hạn bộ nhớ
# # client.close()


In [3]:
save_dir="../../exps"

In [4]:
df = pd.read_csv(f'{save_dir}/train_lag1_pre_processing.csv', index_col=None)

# df = dd.read_csv(f'{save_dir}/train_lag1_pre_processing.csv')
# df = df.compute() 
has_null = df.isnull().values.any()
print("DataFrame có chứa giá trị NaN không?", has_null)

len(df)

DataFrame có chứa giá trị NaN không? True


1651902

In [5]:
df=df.drop(columns=['Unnamed: 0','row_id','date','origin_date','origin_date_right'])
# df.isnull().sum()
df.dropna(inplace=True)
len(df)

1622156

In [6]:
for i in df.columns:
    print(i,end=',')

county,is_business,product_type,is_consumption,prediction_unit_id,lowest_price_per_mwh,highest_price_per_mwh,data_block_id,eic_count,installed_capacity,euros_per_mwh,data_block_id_right,hours_ahead,temperature,dewpoint,cloudcover_high,cloudcover_low,cloudcover_mid,cloudcover_total,10_metre_u_wind_component,10_metre_v_wind_component,direct_solar_radiation,surface_solar_radiation_downwards,snowfall,total_precipitation,hours_ahead_fl,temperature_fl,dewpoint_fl,cloudcover_high_fl,cloudcover_low_fl,cloudcover_mid_fl,cloudcover_total_fl,10_metre_u_wind_component_fl,10_metre_v_wind_component_fl,direct_solar_radiation_fl,surface_solar_radiation_downwards_fl,snowfall_fl,total_precipitation_fl,hours_ahead_fd_7d,temperature_fd_7d,dewpoint_fd_7d,cloudcover_high_fd_7d,cloudcover_low_fd_7d,cloudcover_mid_fd_7d,cloudcover_total_fd_7d,10_metre_u_wind_component_fd_7d,10_metre_v_wind_component_fd_7d,direct_solar_radiation_fd_7d,surface_solar_radiation_downwards_fd_7d,snowfall_fd_7d,total_precipitation_f

In [7]:
X = df.drop('target', axis=1)
y = df['target']

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42, shuffle=False)

In [9]:


num_folds = 10
seed = 7
scorers = {'mae': make_scorer(mean_absolute_error), 'r2': make_scorer(r2_score)}
models = []

# Defining models
models.append(('LR', LinearRegression()))
models.append(('Ridge', Ridge(alpha=1.0)))
models.append(('Lasso', Lasso(alpha=0.1)))
# models.append(('SVR', SVR(kernel='rbf', C=10, gamma=0.1, epsilon=0.1)))
models.append(('DT', DecisionTreeRegressor(random_state=42)))
models.append(('RF', RandomForestRegressor(n_estimators=100, random_state=42)))
models.append(('GBR', GradientBoostingRegressor(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=42)))
models.append(('LGBM', lgb.LGBMRegressor(objective='regression', num_leaves=31, learning_rate=0.1, n_estimators=100)))

# Function to evaluate models
def check_model(name, model, X, y, scoring):
    kfold = KFold(n_splits=num_folds, shuffle=True, random_state=seed)
    cv_results = cross_val_score(model, X, y, cv=kfold, scoring=scoring)
    msg = f"{scoring} {name}: {cv_results.mean():.3f} ({cv_results.std():.3f})"
    print(msg)
    return cv_results

results_mae = []
names_mae = []
results_r2 = []
names_r2 = []

# Evaluating models
for name, model in models:
    print(f"Evaluating {name}...")
    mae_result = check_model(name, model, X_test, y_test, scoring='neg_mean_absolute_error')
    results_mae.append(mae_result)
    names_mae.append(name)
    
    r2_result = check_model(name, model, X_test, y_test, scoring='r2')
    results_r2.append(r2_result)
    names_r2.append(name)

# Plotting MAE
fig_mae = plt.figure()
fig_mae.suptitle('Comparison of MAE among models')
ax_mae = fig_mae.add_subplot(111)
plt.boxplot(results_mae)
ax_mae.set_xticklabels(names_mae)
plt.ylabel('MAE')
plt.show()

# Plotting R2
fig_r2 = plt.figure()
fig_r2.suptitle('Comparison of R2 among models')
ax_r2 = fig_r2.add_subplot(111)
plt.boxplot(results_r2)
ax_r2.set_xticklabels(names_r2)
plt.ylabel('R2 Score')
plt.show()


Evaluating LR...
neg_mean_absolute_error LR: -0.128 (0.002)
r2 LR: 0.886 (0.005)
Evaluating Ridge...
neg_mean_absolute_error Ridge: -0.128 (0.002)
r2 Ridge: 0.886 (0.005)
Evaluating Lasso...
neg_mean_absolute_error Lasso: -0.111 (0.002)
r2 Lasso: 0.880 (0.005)
Evaluating DT...
neg_mean_absolute_error DT: -0.049 (0.001)
r2 DT: 0.968 (0.003)
Evaluating RF...


,county,is_business,product_type,is_consumption,prediction_unit_id,lowest_price_per_mwh,highest_price_per_mwh,data_block_id,eic_count,installed_capacity,...,target_1,day,weekday,month,year,segment,sin(dayofyear),cos(dayofyear),sin(hour),cos(hour)
0,0,-1.088528,1,0,0,-0.423582,-0.380074,122.0,0.462100,-0.067294,...,0.004,1,6,1,2022,0_0_1_0,0.017166,0.999853,0.000000,1.0
1,0,-1.088528,1,1,0,-0.423582,-0.380074,122.0,0.462100,-0.067294,...,412.153,1,6,1,2022,0_0_1_1,0.017166,0.999853,0.000000,1.0
2,0,-1.088528,2,0,1,-0.423582,-0.380074,122.0,-0.407437,-0.545040,...,0.000,1,6,1,2022,0_0_2_0,0.017166,0.999853,0.000000,1.0
3,0,-1.088528,2,1,1,-0.423582,-0.380074,122.0,-0.407437,-0.545040,...,43.871,1,6,1,2022,0_0_2_1,0.017166,0.999853,0.000000,1.0
4,0,-1.088528,3,0,2,-0.423582,-0.380074,122.0,4.355257,2.454894,...,0.610,1,6,1,2022,0_0_3_0,0.017166,0.999853,0.000000,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10443,4,0.918672,1,1,66,-0.523142,-0.449913,125.0,-0.460137,-0.421234,...,194.500,4,2,1,2022,4_1_1_1,0.068615,0.997643,0.866025,-0.5
10444,4,0.918672,3,0,18,-0.523142,-0.449913,125.0,-0.288864,-0.093382,...,0.000,4,2,1,2022,4_1_3_0,0.068615,0.997643,0.866025,-0.5
10445,4,0.918672,3,1,18,-0.523142,-0.449913,125.0,-0.288864,-0.093382,...,528.227,4,2,1,2022,4_1_3_1,0.068615,0.997643,0.866025,-0.5
10446,5,-1.088528,1,0,19,-0.523142,-0.449913,125.0,-0.321801,-0.470492,...,0.000,4,2,1,2022,5_0_1_0,0.068615,0.997643,0.866025,-0.5


In [ ]:
svr =SVR(kernel='rbf', C=10, gamma=0.1, epsilon=0.1)
svr.fit(X_train[:20000], y_train[:20000])

SVR(C=10, gamma=0.1)

In [ ]:
# Train Decision Tree model
dt_model = DecisionTreeRegressor(random_state=42)
dt_model.fit(X_train, y_train)



In [ ]:
importances = dt_model.feature_importances_
indices = np.argsort(importances)[::-1]

In [ ]:
print("Feature ranking:")
for f in range(X_train.shape[1]):
    print(f"{f + 1}. feature {X_train.columns[indices[f]]} ({importances[indices[f]]})")

# Visualize Decision Tree
plt.figure(figsize=(20,10))
plot_tree(dt_model, feature_names=X_train.columns, filled=True, rounded=True)
plt.title("Decision Tree Structure")
plt.show()

In [ ]:
# Train LightGBM model
lgbm_model = lgb.LGBMRegressor(objective='regression', num_leaves=31, learning_rate=0.1, n_estimators=100)
lgbm_model.fit(X_train, y_train)

In [ ]:
# Evaluating SHAP for Decision Tree
explainer_dt = shap.Explainer(dt_model, X_train)
shap_values_dt = explainer_dt(X_test)

# Evaluating SHAP for LightGBM
explainer_lgbm = shap.Explainer(lgbm_model, X_train)
shap_values_lgbm = explainer_lgbm(X_test)

# Evaluating LIME for Decision Tree
lime_explainer = lime.lime_tabular.LimeTabularExplainer(X_train.values, feature_names=X_train.columns, verbose=True, mode='regression')
lime_exp_dt = lime_explainer.explain_instance(X_test.iloc[0].values, dt_model.predict)

# Evaluating LIME for LightGBM
lime_exp_lgbm = lime_explainer.explain_instance(X_test.iloc[0].values, lgbm_model.predict)

In [ ]:


# SHAP summary plot for Decision Tree
shap.summary_plot(shap_values_dt, X_test, plot_type="bar", title="SHAP Decision Tree Feature Importance")

# SHAP summary plot for LightGBM
shap.summary_plot(shap_values_lgbm, X_test, plot_type="bar", title="SHAP LightGBM Feature Importance")

# LIME explanation for Decision Tree
lime_exp_dt.show_in_notebook()

# LIME explanation for LightGBM
lime_exp_lgbm.show_in_notebook()


# Detailed force plot for one sample in the test set
shap.force_plot(explainer_dt.expected_value, shap_values_dt[0,:], X_test.iloc[0,:], matplotlib=True)
shap.force_plot(explainer_lgbm.expected_value, shap_values_lgbm[0,:], X_test.iloc[0,:], matplotlib=True)